In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
import re
import matplotlib.cm as cm
from matplotlib import rcParams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

C:\Users\Mahmoud Montaser\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Mahmoud Montaser\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Mahmoud Montaser\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Read the data 

In [7]:
lastdata = pd.read_csv("lastdata.csv",usecols=["Comments","Dialect"])
lastdata.head()

,Comments,Dialect
0,"['يلا', 'نروح', 'يازعبله', 'ويمكن', 'ربنا', 'ي...",EG
1,"['احنا', 'مش', 'قادرين', 'علي', 'البشر', 'نقدر...",JO
2,"['شو', 'دخل', 'القهوه', 'بالصوره']",SY
3,"['جميله', 'التنافسيه', 'الانديه', 'بالنسبه', '...",QA
4,"['يسولف', 'حاله', 'انا', 'والله', 'الكرونك', '...",JO


## 5- Split The Data :

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(lastdata["Comments"],lastdata["Dialect"], test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train, test_size=0.1, random_state=42)

## 6 - TF-IDF Vectorizer :

In [9]:
vectorizer = TfidfVectorizer(sublinear_tf=True)

In [10]:
vectorizer.fit(X_train.astype("U"))          # Fit the X Xtrain Only 

TfidfVectorizer(sublinear_tf=True)

In [11]:
Xtrain = vectorizer.transform(X_train.astype("U"))    # Transfor the Xtrain-Xval-Xtext
Xval = vectorizer.transform(X_valid.astype("U"))
Xtest = vectorizer.transform(X_test.astype("U"))

In [12]:
## Save the Vectorizer Model : 
filename = 'TFIDF2.pkl'
pickle.dump(vectorizer, open(filename, 'wb'))

In [13]:
## Load the Saved Vectorizer Model to use it in FastAPI : 
filename2 = 'TFIDF2.pkl'
TFIDF = pickle.load(open(filename2, 'rb'))

## 7- Create the Machine Learning Model (  LinearSVM )
A- Create The Model .

B- Evaluate The Model [Check for Accuracy , Precision , Recall , F1 Score For Validation And Test Data ]

C- Save The Machine Learning Model 

## A- Create The Model 

In [14]:
from sklearn.svm import LinearSVC
from sklearn import svm
linSVC = LinearSVC(random_state=42, tol=1e-3)
linSVC.fit(Xtrain, y_train)

LinearSVC(random_state=42, tol=0.001)

In [15]:
y_pred = linSVC.predict(Xval)

## B- Accuracy , Precision , Recall , F1 Score        [ Validation Data ]

In [16]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": round(model_accuracy,1),
                  "precision": round(model_precision* 100,1),
                  "recall": round(model_recall* 100,1),
                  "f1": round(model_f1* 100,1)}
    return model_results

y_pred = linSVC.predict(Xval)
result = calculate_results(y_valid, y_pred)
result

{'accuracy': 53.5, 'precision': 52.9, 'recall': 53.5, 'f1': 52.8}

## B- Accuracy for Testing Data 

In [17]:
accuracy_score(linSVC.predict(Xtest),y_test)

0.5311741872877244

In [18]:
def pred(x):
    return linSVC.predict(vectorizer.transform([x]))
pred("انا اسمي تامر يسطي صباح الفل")

array(['EG'], dtype=object)

In [19]:
## Save the Machine Learning Model to disk: 
filename = 'MachineLearningModelSVC.sav'
pickle.dump(linSVC, open(filename, 'wb'))

## 8)  Deep Learning  Model  :  [ Pre-Trained Model  ( bert - base - arabertv02 - twitter )  ]

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";                      ## To Use the GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

In [ ]:
import pandas as pd
df = pd.read_csv("ForDeepLearning.csv",usecols=["Comments","Dialect"], encoding='utf-8')## Use the Data saved before Tokenization
df.head(3)

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
df = df.dropna()

In [ ]:
from sklearn import preprocessing              #Label Encoding the Target 
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
df['Encoded_Dialect']= label_encoder.fit_transform(df['Dialect'])
 
df['Encoded_Dialect'].unique()

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 64
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM , BertForSequenceClassification 
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02-twitter")

In [ ]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.Dialect.iloc[index])
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.Encoded_Dialect.iloc[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(df[["Comments","Encoded_Dialect","Dialect"]], test_size=0.1, 
                                                  random_state=42, stratify = df["Encoded_Dialect"])

In [ ]:
training_set = Triage(X_train, tokenizer, MAX_LEN)
testing_set = Triage(X_val, tokenizer, MAX_LEN)

## DataLoader :

In [ ]:
from torch.utils.data import Dataset, DataLoader
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

## Creating the Neural Network for Fine Tuning :
-- Neural Network

-- Loss Function and Optimizer

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 
from transformers import BertModel
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained("aubmindlab/bert-base-arabertv02-twitter")
        self.classifier = torch.nn.Linear(768, 18)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        output = self.classifier(pooler)
        return output


In [ ]:
model = BERTClass()
model.to(device)

In [ ]:
for param in model.l1.parameters():
    param.requires_grad = False

params_to_update = []
for name,param in model.named_parameters():
#     print(name)
    if param.requires_grad == True:
        params_to_update.append(param)


In [ ]:
print(len(params_to_update))

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  params_to_update, lr=LEARNING_RATE)

## Fine Tuning the Model :

In [ ]:
# Function to calcuate the accuracy of the model

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

## Validating the Model

In [ ]:
#def valid(model, testing_loader):
 #   model.eval()
  #  n_correct = 0; n_wrong = 0; total = 0
   # with torch.no_grad():
    #    for _, data in enumerate(testing_loader, 0):
     #       ids = data['ids'].to(device, dtype = torch.long)
      #      mask = data['mask'].to(device, dtype = torch.long)
    #        targets = data['targets'].to(device, dtype = torch.long)
     #       outputs = model(ids, mask).squeeze()
       #     loss = loss_function(outputs, targets)
       #      tr_loss += loss.item()
        #    big_val, big_idx = torch.max(outputs.data, dim=1)
       #     n_correct += calcuate_accu(big_idx, targets)

        #    nb_tr_steps += 1
        #    nb_tr_examples+=targets.size(0)
            
         #   if _%5000==0:
          #      loss_step = tr_loss/nb_tr_steps
           #     accu_step = (n_correct*100)/nb_tr_examples
            #    print(f"Validation Loss per 100 steps: {loss_step}")
             #   print(f"Validation Accuracy per 100 steps: {accu_step}")
   # epoch_loss = tr_loss/nb_tr_steps
   # epoch_accu = (n_correct*100)/nb_tr_examples
   # print(f"Validation Loss Epoch: {epoch_loss}")
   # print(f"Validation Accuracy Epoch: {epoch_accu}")
    
   # return epoch_accu

In [ ]:
#print('This is the validation section to print the accuracy and see how it performs')
#print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

#acc = valid(model, testing_loader)
#print("Accuracy on test data = %0.2f%%" % acc)

## Saving the Trained Model Artifacts for inference

In [ ]:
# Saving the files for re-use
torch.save(model.state_dict(), '../models/transformer_wts.pth')
print('All files saved')